ใน ep นี้เราจะมาใช้ tensorboard ให้ในทำ Visualization ข้อมูล Embedding ของหนังจาก MovieLens 100k กัน ส่วนรายละเอียดการเทรนโมเดล Collaborative Filtering ชุดข้อมูล MovieLens ขอให้ดู ep ก่อน ๆ

# 0. Magic Commands


เพิ่ม magic command ของ tensorboard

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

%load_ext tensorboard

ถ้ายังไม่มี tensorboard ให้ install ด้วย cell ด้านล่าง

In [2]:
# !conda install -y -c conda-forge tensorboard 

# 1. Import Library

In [3]:
from fastai import *
from fastai.collab import *
from fastai.tabular import *

import time
import numpy as np
import pandas as pd

from fastai_tensorboard_callback import *

ถ้ายังไม่ได้ติดตั้ง fastai_tensorboard_callback ให้ติดตั้งโดยรัน cell ด้านล่าง

In [4]:
# !pip install git+https://github.com/Pendar2/fastai-tensorboard-callback.git

# 2. ข้อมูล

รัน cell ด้านล่างเพื่อดาวน์โหลด [Movielen-100k](http://files.grouplens.org/datasets/movielens/ml-100k.zip) ไปใส่ไว้ที่ ~/.fastai/data/ml-100k แล้วแตกไฟล์

In [5]:
# ! wget http://files.grouplens.org/datasets/movielens/ml-100k.zip 
# ! mv ./ml-100k.zip ~/.fastai/data
# ! unzip ~/.fastai/data/ml-100k.zip -d ~/.fastai/data

In [6]:
path=Config.data_path()/'ml-100k'

In [7]:
user, item, title = 'userId', 'movieId', 'title'

In [8]:
dataframe_ratings = pd.read_csv(path/'u.data', 
                                delimiter='\t', header=None,
                                names=[user,item,'rating','timestamp'])

# 3. เตรียมข้อมูล



## ข้อมูล Content

ถ้าเราดูแต่ movieId ก็จะไม่รู้ว่าเป็นหนังเรื่องไหน เราจะไปดึงข้อมูล Master ของ Movie มา merge และเพื่อนำไปแสดงบน Tensorboard ด้วย

In [9]:
dataframe_movies = pd.read_csv(path/'u.item',  delimiter='|', encoding='latin-1', header=None,
                    names=[item, 'title', 'date', 'N', 'url', *[f'g{i}' for i in range(19)]])

In [10]:
dataframe_rating_movie = dataframe_ratings.merge(dataframe_movies[[item, title]])

In [11]:
databunch = CollabDataBunch.from_df(dataframe_rating_movie, 
                                    seed=42, valid_pct=0.1,                                     
                                    item_name=title)

สร้างไฟล์ Metadata ชื่อหนังไว้ไปแสดงบน Tensorboard

In [ ]:
dataframe_movies.to_csv('movie_master.tsv', sep = '\t')

# 4. สร้างโมเดล

In [12]:
y_range = [0,5.5]

In [13]:
proj_id = '07b-tensorboard-embedding'
tboard_path = Path('data/tensorboard/' + proj_id)

print(f"tensorboard --logdir={tboard_path}")

tensorboard --logdir=data/tensorboard/07b-tensorboard-embedding


In [14]:
learner = collab_learner(databunch, n_factors=50, 
                         y_range=y_range, wd=1e-1)

สำหรับ clear logs

In [15]:
# ! rm -rf $tboard_path

เราสามารถดูเปิด Tensorboard ได้ 2 ทาง

1. ใน Notebook ให้รัน cell magic command ด้านล่าง

2. นอก Notebook เปิด Shell ใหม่ แล้วรันคำสั่ง tensorboard --logdir=./data/tensorboard/07a-tensorboard-mnist แล้วเปิดลิงค์ http://localhost:6006

*เราอาจจะต้องรอให้เทรนผ่าน Epoch แรกไปก่อน ถึงจะเห็นกราฟ*

In [ ]:
%tensorboard --logdir=$tboard_path

![tensorboard screenshot](https://www.bualabs.com/wp-content/uploads/2019/08/bualabs-tensorboard-embedding-movielens-100k-04.png)

# 5. เริ่มต้นเทรนโมเดล

เทรนด้วยค่า Default ไป 5 Cycle สามารถใช้ Learning Rate สูง ๆ ได้เลย

In [17]:
learner.fit_one_cycle(5, 5e-3, callbacks=[TensorboardLogger(learner,  
                                    f'Learner{time.time()}',
                                    path=tboard_path)])

epoch,train_loss,valid_loss,time
0,0.938870,0.949390,00:15
1,0.878606,0.880336,00:18
2,0.766966,0.839303,00:16
3,0.620651,0.817506,00:15
4,0.491416,0.815580,00:15


## สำเร็จแล้ว ##

เราสามารถใช้ Tensorboard ดู Embedding ภายในโมเดล ไม่ว่าจะใช้ Dimensional Reduction แบบ PCA, T-SNE, UMAP ไปถึงนำไป Present ได้อย่างสวยงาม


In [18]:
learner.save("07b-tensorboard-embedding")

In [19]:
learner.load("07b-tensorboard-embedding");

# 6. ดูผลลัพธ์

เปิดลิงค์ http://localhost:6006 ดูในแท็บ Projector

In [20]:
learner.model

EmbeddingDotBias(
  (u_weight): Embedding(944, 50)
  (i_weight): Embedding(1654, 50)
  (u_bias): Embedding(944, 1)
  (i_bias): Embedding(1654, 1)
)

# 7. สรุป


1. เราสามารถเรียกใช้ Tensorboard แสดง Embedding สวยงาม ได้ด้วยโค้ด callback ไม่กี่บรรทัด
2. เราต้องเตรียม Metadata เช่น ชื่อ รูปประกอบ เพิ่มให้กับ Tensorboard ไม่งั้นจะเห็นแต่ ตัวเลข, ID ทำให้ดูไม่รู้เรื่่อง
3. แต่การตีความ Chart ของ Embedding จำเป็นต้องมีความรู้ความเข้าใจ ข้อมูลแบบหลายมิติ และการลดมิติ Dimensionality Reduction ที่เราจะอธิบายต่อไป

# Credit 

* https://github.com/Pendar2/fastai-tensorboard-callback
* [FastAI: Practical Deep Learning for Coders, v3 - Lesson 4](https://course.fast.ai/videos/?lesson=4)
* [Movielens-100k](https://grouplens.org/datasets/movielens/100k/)
* F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages. DOI=http://dx.doi.org/10.1145/2827872
